In [3]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression

import os

In [4]:
cwd = os.getcwd()

In [18]:
df_seeds = pd.read_csv(cwd + '/data/MNCAATourneySeeds.csv')
df_tour = pd.read_csv(cwd + '/data/MNCAATourneyCompactResults.csv')

In [9]:
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [19]:
df_tour.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [10]:
def seed_to_int(seed):
    s_int = int(seed[1:3])
    return s_int

In [11]:
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)

In [12]:
df_seeds.head()

,Season,Seed,TeamID,seed_int
0,1985,W01,1207,1
1,1985,W02,1210,2
2,1985,W03,1228,3
3,1985,W04,1260,4
4,1985,W05,1374,5


In [14]:
df_seeds.drop(labels=['Seed'], inplace=True, axis=1)

In [15]:
df_seeds.head()

,Season,TeamID,seed_int
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [20]:
df_tour.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [21]:
df_tour.drop(labels=['DayNum'], inplace=True, axis=1)
df_tour.head()

,Season,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,1116,63,1234,54,N,0
1,1985,1120,59,1345,58,N,0
2,1985,1207,68,1250,43,N,0
3,1985,1229,58,1425,55,N,0
4,1985,1242,49,1325,38,N,0
